# Kepler Problem and Orbital Mechanics

This notebook demonstrates the solution of Kepler's equation and visualization of elliptical orbits using the AstroLibrary.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src directory to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from classical import solve_kepler, orbital_elements_to_state, orbital_period, true_anomaly_from_eccentric
from utils import deg_to_rad, rad_to_deg, AU, solar_mass, G

plt.style.use('default')
%matplotlib inline

## Solving Kepler's Equation

Kepler's equation relates the mean anomaly M to the eccentric anomaly E:

$$E - e \sin E = M$$

where e is the eccentricity.

In [ ]:
# Demonstrate Kepler equation solution for different eccentricities
M_values = np.linspace(0, 2*np.pi, 100)
eccentricities = [0.0, 0.2, 0.5, 0.8]

plt.figure(figsize=(12, 8))

for i, e in enumerate(eccentricities):
    E_values = [solve_kepler(M, e) for M in M_values]
    
    plt.subplot(2, 2, i+1)
    plt.plot(rad_to_deg(M_values), rad_to_deg(E_values), 'b-', linewidth=2)
    plt.plot(rad_to_deg(M_values), rad_to_deg(M_values), 'r--', alpha=0.7, label='E = M (e=0)')
    plt.xlabel('Mean Anomaly M (degrees)')
    plt.ylabel('Eccentric Anomaly E (degrees)')
    plt.title(f'Kepler\'s Equation Solution (e = {e})')
    plt.grid(True, alpha=0.3)
    plt.legend()

plt.tight_layout()
plt.show()

## Earth-Sun System Parameters

Let's use realistic parameters for the Earth-Sun system to demonstrate orbital calculations.

In [ ]:
# Earth-Sun system parameters
a_earth = 1.0 * AU  # 1 AU semi-major axis
e_earth = 0.0167    # Earth's eccentricity
M_sun = solar_mass  # Solar mass
mu_sun = G * M_sun  # Standard gravitational parameter

# Calculate orbital period
T_earth = orbital_period(a_earth, mu_sun)
print(f"Earth's orbital period: {T_earth / (365.25 * 24 * 3600):.3f} years")
print(f"Earth's orbital period: {T_earth / (24 * 3600):.1f} days")

## Visualizing Elliptical Orbits

Now let's create orbital trajectories for different eccentricities and visualize them.

In [ ]:
def plot_orbit(a, e, mu, label, color='blue', num_points=500):
    """Plot an elliptical orbit"""
    M_vals = np.linspace(0, 2*np.pi, num_points)
    positions = []
    
    for M in M_vals:
        pos, vel = orbital_elements_to_state(a, e, M, mu)
        positions.append(pos)
    
    positions = np.array(positions)
    x_coords = positions[:, 0] / AU  # Convert to AU for plotting
    y_coords = positions[:, 1] / AU
    
    plt.plot(x_coords, y_coords, color=color, linewidth=2, label=label)
    return x_coords, y_coords

# Plot orbits with different eccentricities
plt.figure(figsize=(12, 10))

colors = ['blue', 'green', 'red', 'purple']
eccentricities = [0.0, 0.2, 0.5, 0.7]

for e, color in zip(eccentricities, colors):
    plot_orbit(a_earth, e, mu_sun, f'e = {e}', color)

# Add the Sun at the focus
plt.scatter([0], [0], color='orange', s=200, label='Sun', zorder=5)

plt.axis('equal')
plt.xlabel('x (AU)')
plt.ylabel('y (AU)')
plt.title('Elliptical Orbits with Different Eccentricities')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(-1.8, 1.8)
plt.ylim(-1.8, 1.8)
plt.show()

## Orbital Velocity Variation

Let's examine how orbital velocity varies around an elliptical orbit (Kepler's Second Law).

In [ ]:
# Calculate velocity variation for an eccentric orbit
e_demo = 0.5
M_vals = np.linspace(0, 2*np.pi, 100)
velocities = []
distances = []
true_anomalies = []

for M in M_vals:
    pos, vel = orbital_elements_to_state(a_earth, e_demo, M, mu_sun)
    velocities.append(np.linalg.norm(vel))
    distances.append(np.linalg.norm(pos))
    
    # Calculate true anomaly
    E = solve_kepler(M, e_demo)
    nu = true_anomaly_from_eccentric(E, e_demo)
    true_anomalies.append(nu)

velocities = np.array(velocities) / 1000  # Convert to km/s
distances = np.array(distances) / AU     # Convert to AU
true_anomalies = np.array(true_anomalies)

# Plot velocity and distance vs true anomaly
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

ax1.plot(rad_to_deg(true_anomalies), velocities, 'b-', linewidth=2)
ax1.set_xlabel('True Anomaly (degrees)')
ax1.set_ylabel('Orbital Velocity (km/s)')
ax1.set_title(f'Orbital Velocity Variation (e = {e_demo})')
ax1.grid(True, alpha=0.3)
ax1.axvline(0, color='red', linestyle='--', alpha=0.7, label='Periapsis')
ax1.axvline(180, color='green', linestyle='--', alpha=0.7, label='Apoapsis')
ax1.legend()

ax2.plot(rad_to_deg(true_anomalies), distances, 'r-', linewidth=2)
ax2.set_xlabel('True Anomaly (degrees)')
ax2.set_ylabel('Distance from Sun (AU)')
ax2.set_title(f'Distance Variation (e = {e_demo})')
ax2.grid(True, alpha=0.3)
ax2.axvline(0, color='red', linestyle='--', alpha=0.7, label='Periapsis')
ax2.axvline(180, color='green', linestyle='--', alpha=0.7, label='Apoapsis')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Maximum velocity (at periapsis): {velocities.max():.2f} km/s")
print(f"Minimum velocity (at apoapsis): {velocities.min():.2f} km/s")
print(f"Velocity ratio: {velocities.max()/velocities.min():.2f}")

## Planetary System Comparison

Let's compare the orbits of different planets in our solar system.

In [ ]:
# Solar system data (approximate)
planets = {
    'Mercury': {'a': 0.387*AU, 'e': 0.2056, 'color': 'gray'},
    'Venus':   {'a': 0.723*AU, 'e': 0.0067, 'color': 'orange'},
    'Earth':   {'a': 1.000*AU, 'e': 0.0167, 'color': 'blue'},
    'Mars':    {'a': 1.524*AU, 'e': 0.0934, 'color': 'red'},
}

plt.figure(figsize=(14, 10))

for name, data in planets.items():
    a, e, color = data['a'], data['e'], data['color']
    plot_orbit(a, e, mu_sun, f'{name} (e={e:.3f})', color)

# Add the Sun
plt.scatter([0], [0], color='yellow', s=300, label='Sun', zorder=5, edgecolors='black')

plt.axis('equal')
plt.xlabel('x (AU)')
plt.ylabel('y (AU)')
plt.title('Inner Solar System Orbits')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.show()

# Calculate and display orbital periods
print("Orbital Periods:")
print("-" * 30)
for name, data in planets.items():
    a = data['a']
    T = orbital_period(a, mu_sun)
    T_years = T / (365.25 * 24 * 3600)
    print(f"{name:8s}: {T_years:.2f} years")

## Transfer Orbits

Let's demonstrate a Hohmann transfer orbit between two circular orbits.

In [ ]:
# Hohmann transfer between Earth and Mars orbits
r1 = 1.0 * AU  # Earth orbit radius
r2 = 1.524 * AU  # Mars orbit radius

# Transfer orbit parameters
a_transfer = (r1 + r2) / 2
e_transfer = (r2 - r1) / (r2 + r1)

print(f"Transfer orbit semi-major axis: {a_transfer/AU:.3f} AU")
print(f"Transfer orbit eccentricity: {e_transfer:.3f}")

# Transfer time (half period)
T_transfer = orbital_period(a_transfer, mu_sun)
transfer_time = T_transfer / 2
transfer_days = transfer_time / (24 * 3600)

print(f"Transfer time: {transfer_days:.0f} days ({transfer_days/30:.1f} months)")

# Plot the transfer
plt.figure(figsize=(10, 10))

# Plot circular orbits
theta = np.linspace(0, 2*np.pi, 100)
x1 = r1/AU * np.cos(theta)
y1 = r1/AU * np.sin(theta)
x2 = r2/AU * np.cos(theta)
y2 = r2/AU * np.sin(theta)

plt.plot(x1, y1, 'b-', linewidth=2, label='Earth orbit')
plt.plot(x2, y2, 'r-', linewidth=2, label='Mars orbit')

# Plot transfer orbit
plot_orbit(a_transfer, e_transfer, mu_sun, 'Hohmann transfer', 'green')

# Mark planets at transfer points
plt.scatter([r1/AU], [0], color='blue', s=100, label='Earth at departure', zorder=5)
plt.scatter([-r2/AU], [0], color='red', s=100, label='Mars at arrival', zorder=5)

# Add the Sun
plt.scatter([0], [0], color='yellow', s=300, label='Sun', zorder=5, edgecolors='black')

plt.axis('equal')
plt.xlabel('x (AU)')
plt.ylabel('y (AU)')
plt.title('Hohmann Transfer from Earth to Mars')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.show()